# LSTMega

Este projeto é um modelo de rede neural recorrente utilizando a arquitetura *Long Short-Term Memory* &ndash; LSTM para geração automatizada de textos.

Ele está baseado em uma implementação da biblioteca [Keras](https://keras.io/) e foi iniciado a partir de um exemplo da utilização de redes neurais recorrentes com LSTM, onde este foi disponibilizado na própria documentação do Keras e pode ser acessado [aqui](https://keras.io/examples/lstm_text_generation/).

Todo o projeto e histórico de mudanças está disponível no repositório [lstmega](https://github.com/mlc2307/lstmega) no GitHub.

As bibliotecas base deste projeto são a Keras e o [TensorFlow](https://www.tensorflow.org/). Elas serão carregadas durante todo o processo.

É valido lembrar que nem todas as bibliotecas são importadas no exemplo original.

In [ ]:
# Deve estar na primeira linha.
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.client import device_lib as dlib

Retorna informações sobre o computador.

In [ ]:
# Informa a GPU que será utilizada no treinamento.
tf.test.gpu_device_name()

In [ ]:
# Lista os dispositivos locais.
dlib.list_local_devices()

Bibliotecas que são utilizadas no decorrer da aplicação.

In [ ]:
# Ferramentas para leitura do texto.
from keras.utils.data_utils import get_file
from os.path import basename
import io

# Para a geração dos arrays a partir do texto.
import numpy as np

# Oferece opções de redes neurais.
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

# Bibliotecas para a função on_epoch_end.
import random, sys

# Opções de configuração da compilação.
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

O primeiro passo, após importarmos as bibliotecas acima, é informar à rede neural o texto que será utilizado para o aprendizado do modelo.

Neste caso, assim como no exemplo original, o texto está disponível remotamente e pode ser acessado através deste [link](https://s3.amazonaws.com/text-datasets/nietzsche.txt). Ao executar a célula abaixo será feito o download automaticamente.

In [ ]:
# Origem remota do arquivo de texto utilizado no aprendizado do modelo.
origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt"

path = get_file(basename(origin), origin)

# Faz a leitura do arquivo, deixando todos os caracteres minúsculos.
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
    
print("Tamanho do arquivo:", len(text))

# Verifica quantos caracteres diferentes existem no texto.
chars = sorted(list(set(text)))

print("Total de caracteres:", len(chars))

# Faz a indexação dos caracteres.
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

A seguir, o algoritmo fatia o texto em sequências semi-redundantes, cuja quantidade de caracteres destas sequências é definida pela variável `maxlen`.

Nada desta seção foi alterada em relação ao código-fonte original.

In [ ]:
# Tamanho das fatias do texto.
maxlen = 40

step = 3

sentences = []
next_chars = []

# Processo responsável por fatiar o texto em sequências.
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print("Total de sequências:", len(sentences))

print("\nVetorizando as sequências...")

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        
    y[i, char_indices[next_chars[i]]] = 1

print("Sequências vetorizadas!")

Na próxima etapa vem a definição e construção do modelo de rede neural. Como se trata de uma rede neural recorrente do tipo LSTM, deve ser usada a implementação que a Keras oferece para se trabalhar com este tipo de modelo.

Esta parte do código foi alterada para que retorne informações relevantes.

In [ ]:
print("Construindo o modelo...")

model = Sequential()

# A camada de entrada é do tipo LSTM.
model.add(LSTM(128, input_shape=(maxlen, len(chars))))

# Adiciona uma camada do tipo Dense com 24 neurônios e 0.2 de dropout.
model.add(Dense(24, activation="relu"))
model.add(Dropout(0.2))

# Adiciona uma camada do tipo Dense com 32 neurônios e 0.3 de dropout.
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))

# Camada de saída.
model.add(Dense(len(chars), activation="softmax"))

print("Modelo construído!\n")

model.summary()

Antes de partir para a compilação e treinamento, serão definidas duas funções que ajudarão nos processos. Elas são definidas, também, no exemplo original já citado.

A função `sample` amostra um índice de uma matriz de probabilidade.

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    
    exp_preds = np.exp(preds)
    
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

Já a função `on_epoch_end` simplesmente mostra um *feedback* com algumas informações ao final de cada época.

In [ ]:
def on_epoch_end(epoch, _):
    print("\n\nGerando texto após época #{:d}...".format(epoch + 1))

    start_index = random.randint(0, len(text) - maxlen - 1)
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("\n{:->10s}".format(""))
        print("Diversidade:", diversity)

        generated = ""
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        
        print("Gerando com a seed \"" + sentence + "\"\n")
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        
        print()
    
    print("{:s}{:->20s}".format("\n" * 3, ""))

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Agora pode ser feita a configuração da compilação e treinamento deste modelo.

Neste trecho vários parâmetros foram alterados do original, como o otimizador, o tamanho do *batch*, entre outros fatores.

O mais importante a se notar, que inclusive é citado no texto da página do exemplo original, é que o modelo provavelmente irá gerar um texto coeso por volta da vigésima época, apenas.

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    metrics=['accuracy'],
    optimizer=RMSprop()
)

model.fit(
    x, y,
    batch_size=96,
    epochs=64,
    callbacks=[print_callback]
)

A cada época será gerado um texto usando diferentes *diversidades*. Quanto maior a diversidade, maior a "mistura" de caracteres empregada na geração, e, quanto mais avançada a época, mais coeso é o texto.